# Initialization
Shakespeare example

In [ ]:
!rm -rf /content/*

In [ ]:
#@markdown ## Clonamos el repositorio de nanoGPT y las librerías necesarias
%cd /content/
!git clone https://github.com/jordiluque/nanoGPT

%cd nanoGPT

!pip3 install -r requirements.txt --quiet
!pip3 install transformers --quiet

/content
Cloning into 'nanoGPT'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 106 (delta 36), reused 91 (delta 24), pack-reused 0
Receiving objects: 100% (106/106), 3.15 MiB | 16.27 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/nanoGPT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.8 MB/s eta 0:00:00
  

# Curado de datos y preparación de tokens

Este script de python se descarga un ejemplo de un libro de Shakespeare

In [ ]:
!cat /content/nanoGPT/data/shakespeare/prepare.py

## Descargamos y curamos los datos

In [ ]:
%cd /content/nanoGPT/
#!pip install codecarbon --quiet
#!codecarbon init
#!echo "log_level = CRITICAL" >> .codecarbon.config
#!echo "save_to_api = True" >> .codecarbon.config
from data.shakespeare.prepare import Dataset
import logging, os

# Set logging to debug
logging.basicConfig(level=logging.DEBUG)
# Example using shakespeare
ds = Dataset()
ds.fetch()
ds.save('input.txt')
ds.load('input.txt')
ds.parse()
ds.export('./data/shakespeare/')

/content/nanoGPT
train has 301966 tokens
val has 36059 tokens


Aquí podemos ver un pequeño ejemplo de
cómo son los datos de entrenamiento

In [ ]:
!cat input.txt | head -100

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.




## Dividimos el texto en tokens

In [ ]:
#@markdown ## Un diccionario formado por tokens, en este caso,
#@markdown ## de carácteres en lugar de palabras
!python data/shakespeare_char/prepare.py

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


# Entrenamiento

Utilizando como corpus de entrenamiento una novela de Shakespeare y un diccionario vamos a entrenar un modelo de nano-GPT desde cero.

El diccionario lo hemos obtenido en el paso anterior  de carácteres que hemos obtenido mirando las diferentes ocurrencias en el corpus de entrenamiento: **!$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz**
Estos son nuestros tokens de entrenamiento. Tokenizar consiste en traducir un texto utilizando en, este caso, estos tokens.

## Entrenamiento con carácteres

Entrenamos un mini-GPT desde cero con 100 iteraciones

In [ ]:

!python train.py config/train_shakespeare_char.py --max_iters=100 --eval_iters=20 --eval_interval=50 --out_dir='out-shakespeare-char-100' --batch_size=32

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

### Sintetizando una respuesta (no prompting)

El resultado es realmente pobre. El modelo intenta generar tokens (los del diccionario anterior) pero con tan pocas iteraciones de entrenamiento, sólo 100, genera texto **sin sentido**.

In [ ]:
!python sample.py --out_dir='out-shakespeare-char-100' --device=cuda --num_samples=1 

Overriding: out_dir = out-shakespeare-char-100
Overriding: device = cuda
Overriding: num_samples = 1
number of parameters: 10.65M
Loading meta from data/shakespeare_char/meta.pkl...

And the RO:
Thowilen is s, be mat sen bobe t eranthath my calatanss ar hapat us ce he.
War d las aten wice my.

Thedatom oroupenowhavetof wath bl t st nd ll, bes iree sengcin lat Her d ov ts, and t l nghire transesel lind pe l.
Hashe ce wiry pthar aissplle y hellinde n Badod sigod whomery wod methake o Windo when piiche the m fouris hane hiend t so mower;
Wh

An ad nthrurt f s t; wor t mef t te maleronthefaf Pre?

Wheam.
Whe--sean stied isar adsal the ERDLAnd fin cour ay aney IOUS:

IOfo t ce wi
---------------


### Entrenamiento con 200 iteraciones
Mas iteraciones de entrenamiento (200 iteraciones)
Continuamos entrenando el nano-GPT, partiendo del anterior, hasta las 200 iteraciones.


In [ ]:
!python train.py config/train_shakespeare_char.py --init_from="resume" --max_iters=200 \
  --eval_iters=20 --eval_interval=100 --out_dir='out-shakespeare-char-100' --batch_size=32

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

#### Sintetizando una respuesta (no prompting)

El resultado es un poco mejor. El modelo genera tokens (los del diccionario de carácteres) Sin embargo, con sólo 200 iteraciones de entrenamiento, el texto sigue sin tener sentido, pero empezamos a ver un cierto patrón, el **diálogo**.



In [ ]:
!python sample.py --device="cuda" --out_dir='out-shakespeare-char-100' --num_samples=1

Overriding: device = cuda
Overriding: out_dir = out-shakespeare-char-100
Overriding: num_samples = 1
number of parameters: 10.65M
Loading meta from data/shakespeare_char/meta.pkl...

CANUDIIILA:
My will th la, ber

Hiser bobe ding Shir--
SABRD:
Tand bar hipar ut he hent?
F dilasoate ar ce my.


DUMatom INour
Y:
GALAUCf is he me mil ndilll, es iree sengein lat Herid ov the andove pomanonderansesel lind te l.




CHES:
Any:
Shar aise hewhy.
Herinde n BESpetelaves homy y wod methake o Windo wher pis al the m dourise we shire s poousower;
We

ANROMIONCHET:
A sor; irist me me te maleronthe ar Prerd my o myourdoure!



Sou sar adat Wise ERD sthinin cour ay aney Ire thechan t ce hy
---------------


### Entrenamos un nano-GPT con 1000 iteraciones
![An image](https://github.com/karpathy/nanoGPT/blob/master/assets/gpt2_124M_loss.png?raw=true)

Normalmente, cuando se entrena un modelo de aprendizaje automático se validan los resultados del entrenamiento a cada paso (step) o iteración contra una loss (función de pérdidas) que se intenta mejorar. La siguiente gráfica muestra un comportamiento clásico durante el entrenamiento, en dónde, se intenta minimizar esa loss en el conjunto de validación.


In [ ]:
!python train.py config/train_shakespeare_char.py --max_iters=1000 --init_from="resume" --log_interval=100 --eval_iters=20 --eval_interval=500 --out_dir='out-shakespeare-char-100' --batch_size=32

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

#### Sintetizando una respuesta (no prompting)

El resultado es un mucho mejor. El modelo intenta generar tokens (los del diccionario de carácteres) con 1000 iteraciones de entrenamiento. El texto ya tiene sentido y los carácteres (tokens) que nuestro modelo genera empiezan a describir las regularidades del lenguaje trnasformándose en palabras inglesas.

In [ ]:
!python sample.py --device="cuda" --out_dir='out-shakespeare-char-100' --num_samples=1

Overriding: device = cuda
Overriding: out_dir = out-shakespeare-char-100
Overriding: num_samples = 1
number of parameters: 10.65M
Loading meta from data/shakespeare_char/meta.pkl...

And they brince.

SOMERSET:
Were is thy be does angry.

MENENIUS:
I am it is us come.

SICINIUS:
I endreat my father, an I must ones,
to that I command did that misery bothing at Here;
White and him possion to be pelplance, and thus wond my soul
That should fan'd and nor ope to the hopes of demother
To Warwick thy city, the more rive and
To curs of the house
To kind thruft for treason! my lord, I look the paint of my off,
Her son! whom didst advant, and he mothing straight no vex'd
Whose the wou
---------------


## Finetuning (adaptación) con palabras

Un modelo pre-entrenado como GTP-2, entrenado con un corpus inmenso, se adapta o finetunea a la novela de Shakespeare. Ahora en lugar de carácteres, nuestro diccionario de **tokens** son palabras. El entrenamiento, en este caso, consiste no parte de cero, si no que aprovecha un modelo ya entrenado como GPT-2. Con palabras, el modelo sintetiza más rápidamente el lenguaje y encuentra correlaciones de manera mas sencilla (no ha de aprender a formar palabras). La síntesis genera directamente las palabras **tokens** del diccionario, en lugar de carácteres.

In [ ]:
#@markdown ## Un diccionario formado por tokens, en este caso,
#@markdown ## de palabras en lugar de carácteres.
#@markdown ## Fine-tuneamos un GPT-2 de tamaño medio con 40 iteraciones
%cd /content/nanoGPT/
from data.shakespeare.prepare import Dataset
import logging, os

# Set logging to debug
logging.basicConfig(level=logging.DEBUG)
# Example using shakespeare
ds = Dataset()
ds.fetch()
ds.save('input.txt')
ds.load('input.txt')
ds.parse()
ds.export('./data/shakespeare/')
!python train.py config/finetune_shakespeare.py --max_iters=40

/content/nanoGPT
train has 301966 tokens
val has 36059 tokens
Overriding config with config/finetune_shakespeare.py:
import time

out_dir = 'out-shakespeare'
eval_interval = 5
eval_iters = 40
wandb_log = False # feel free to turn on
wandb_project = 'shakespeare'
wandb_run_name = 'ft-' + str(time.time())

dataset = 'shakespeare'
init_from = 'gpt2' # this is the largest GPT-2 model

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 1
gradient_accumulation_steps = 32
max_iters = 20

# finetune at constant LR
learning_rate = 3e-5
decay_lr = False

Overriding: max_iters = 40
tokens per iteration will be: 32,768
Initializing from OpenAI GPT-2 weights: gpt2
loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
numbe

#### Sintetizando una respuesta con palabras (no prompting)

Resultado con 40 iteraciones y adaptando un GPT2 pre-entrenado
El resultado con palabras y el fine-tuning con sólo 40 iteraciones ya es muy bueno. Mucho mejor que con 40 iteraciones entrenando desde cero y/o empleando un diccionario de carácteres.

In [ ]:
!python sample.py --device="cuda" --out_dir=out-shakespeare --num_samples=1 --max_new_tokens=100

Overriding: device = cuda
Overriding: out_dir = out-shakespeare
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
number of parameters: 123.65M
No meta.pkl found, assuming GPT-2 encodings...

- * * All creatures that are created in the image of God are created in the image of man.

NASB.: And now again the LORD spoke to the sons of men; but he spoke not.

LORD:
That is he did command you,
As God himself commanded you,
And ordained you man,
So that you may know how I am made.

NASB.:
I heard you, therefore, call me Lord
And order you to
---------------


#### Sintetizando una respuesta con palabras (prompting)

Resultado con 40 iteraciones y adaptando un GPT2 pre-entrenado
Ahora probamos a sesgar la respuesta de la síntesis preguntando por algo. 
Utilizando la técnica de "Prompting" preguntamos al modelo como si fueramos el rey Richard, "KING RICHARD II: 
        What is the answer to life, the universe, and everything?"
Pedimos por dos ejemplos (samples) **--num_samples=2** que el modelo nos devuelve separados por una línea de guiones.

In [ ]:
!python sample.py \
    --device="cuda" \
    --out_dir=out-shakespeare \
    --start="KING RICHARD II: \
        What is the answer to life, the universe, and everything?" \
    --num_samples=2 --max_new_tokens=100 --device=cuda

Overriding: device = cuda
Overriding: out_dir = out-shakespeare
Overriding: start = KING RICHARD II:          What is the answer to life, the universe, and everything?
Overriding: num_samples = 2
Overriding: max_new_tokens = 100
Overriding: device = cuda
number of parameters: 123.65M
No meta.pkl found, assuming GPT-2 encodings...
KING RICHARD II:          What is the answer to life, the universe, and everything?

MARY LUCIFERUS:    
Sir, if you will, you will meet me in the presence of the dead,
by the good and the unjust, and by the wise and by the blind.

CUT TO:

JOHN EDWARD IV:    
I am dead; it is my nature to die.

JOHN EDWARD IV:       
Why, if I am dead,
---------------
KING RICHARD II:          What is the answer to life, the universe, and everything? What is the answer to our souls, angels, and all that is in heaven? What is the answer to our bodies, and what is the answer to earth? What is the answer to all things, and what is the answer to all things? What is the answer to 

# Entrenamiento de una novela en castellano

En este caso vamos a realizar el mismo ejercico con una novela de Calderón de la Barca. ¿¿Con castellano?? Sí a pesar de ser entrenado con un corpus mayoritariamente en inglés, GPT2 es capaz de aprender las regularidades de otras lenguas. Los sueños, sueños son... 

## Curado de datos y preparación de tokens

Este script de python se descarga un ejemplo de un libro de Calderón de la Barca, "La vida es sueño"

In [ ]:
from data.calderon.prepare import Dataset
import logging, os

# Set logging to debug
logging.basicConfig(level=logging.DEBUG)
# Example using shakespeare
ds = Dataset()
ds.load('./data/calderon/cap1-4.txt')
ds.parse()
ds.export('./data/calderon/')

train has 799499 tokens
val has 86982 tokens


In [ ]:
#@markdown ## ¿Cómo son los datos?
!cat ./data/calderon/cap1-4.txt | head -100

﻿ESCENA PRIMERA.

ROSAURA, CLARIN.

_(Rosaura vestida de hombre aparece en lo alto de las peñas, y baja á
lo llano; tras ella viene Clarin.)_

ROSAURA.

  Hipogrifo violento
  Que corriste parejas con el viento,
  ¿Dónde rayo sin llama,
  Pájaro sin matiz, pez sin escama,
  Y bruto sin instinto
  Natural, al confuso laberinto
  Destas desnudas peñas
  Te desbocas, arrastras y despeñas?
  Quédate en este monte,
  Donde tengan los brutos su Faetonte;
  Que yo, sin más camino
  Que el que me dan las leyes del destino.
  Ciega y desesperada
  Bajaré la aspereza enmarañada
  Deste monte eminente,
  Que arruga al sol el ceño de su frente.
  Mal, Polonia, recibes
  A un extranjero, pues con sangre escribes
  Su entrada en tus arenas,
  Y apénas llega, cuando llega á penas.
  Bien mi suerte lo dice;
  ¿Mas dónde halló piedad un infelice?

CLARIN.

  Dí dos, y no me dejes
  En la posada á mí cuando te quejes;
  Que si dos hemos sido
  Los que de nuestra patria hemos salido
  A probar aventuras,

## Finetuning (adaptación) con palabras

Un modelo pre-entrenado como GTP-2, entrenado con un corpus inmenso, se adapta o fine-tunea a la novela La vida es sueño. Nuestro diccionario de **tokens** son palabras. El entrenamiento, en este caso, no parte de cero, si no que aprovecha un modelo ya entrenado como GPT-2. Ahora el diccionario de palabras se extrae del propio entrenamiento y son palabras en castellano. 

### Adaptando a castellano con 40 iteraciones un GPT2 pre-entrenado

In [ ]:
!python train.py config/finetune_calderon.py --max_iters=40 --eval_interval=5 --eval_iters=40 --device='cuda'

Overriding config with config/finetune_calderon.py:
import time

out_dir = 'out-calderon'
eval_interval = 100
eval_iters = 500
wandb_log = False # feel free to turn on
wandb_project = 'calderon'
wandb_run_name = 'ft-' + str(time.time())

dataset = 'calderon'
init_from = 'gpt2' # this is the largest GPT-2 model

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 2
gradient_accumulation_steps = 32
max_iters = 200

# finetune at constant LR
learning_rate = 3e-5
decay_lr = False

Overriding: max_iters = 40
Overriding: eval_interval = 5
Overriding: eval_iters = 40
Overriding: device = cuda
tokens per iteration will be: 65,536
Initializing from OpenAI GPT-2 weights: gpt2
loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rat

### Sintetizando una respuesta en castellano con palabras (prompting)

Resultado con 40 iteraciones y adaptando a castellano un GPT2 pre-entrenado. Sesgamos la respuesta de la síntesis preguntando por algo, utilizando la técnica de "Prompting" preguntamos al modelo como si fueramos el Demonio, 

DEMONIO.
        ¿Cuál es la respuesta a la vida, el universo y a todo?

Pedimos por dos ejemplos (samples) que el modelo nos devuelve separados por una línea de guiones.

In [ ]:
!python sample.py \
    --device=cuda \
    --out_dir=out-calderon \
    --start="DEMONIO. \
        ¿Cuál es la respuesta a la vida, el universo y a todo?" \
    --num_samples=2 --max_new_tokens=100 

Overriding: out_dir = out-calderon
Overriding: start = DEMONIO.          ¿Cuál es la respuesta a la vida, el universo y a todo?
Overriding: num_samples = 2
Overriding: max_new_tokens = 100
Overriding: device = cuda
number of parameters: 123.65M
No meta.pkl found, assuming GPT-2 encodings...
DEMONIO.          ¿Cuál es la respuesta a la vida, el universo y a todo?

MARCOS.
                    Este y el dolor, espero y pues, años con el mis cuatro y el muyo.

ESCENA XII.

HONOR, CALVIN.

                          Y á
---------------
DEMONIO.          ¿Cuál es la respuesta a la vida, el universo y a todo?

YES. 

D. O.M., cárin.                    Que uní, la muerte, para te gusta, cuanto,
  Pues en deja, el mano ha de un verdad
  ¿Por tener, cuando?

D. 

D. O.M., cárin. 
---------------


In [ ]:
!python train.py config/finetune_calderon.py --init_from='resume' --device='cuda' \
  --max_iters=400 --eval_interval=20 --eval_iters=40 

Overriding config with config/finetune_calderon.py:
import time

out_dir = 'out-calderon'
eval_interval = 100
eval_iters = 500
wandb_log = False # feel free to turn on
wandb_project = 'calderon'
wandb_run_name = 'ft-' + str(time.time())

dataset = 'calderon'
init_from = 'gpt2' # this is the largest GPT-2 model

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 2
gradient_accumulation_steps = 32
max_iters = 200

# finetune at constant LR
learning_rate = 3e-5
decay_lr = False

Overriding: init_from = resume
Overriding: device = cuda
Overriding: max_iters = 400
Overriding: eval_interval = 20
Overriding: eval_iters = 40
tokens per iteration will be: 65,536
Resuming training from out-calderon
number of parameters: 123.65M
num decayed parameter tensors: 50, with 124,318,464 parameters


### Sintetizando una respuesta en castellano con palabras con 400 iteraciones (prompting)

Resultado con 400 iteraciones y adaptando a castellano un GPT2 pre-entrenado. Sesgamos la respuesta de la síntesis preguntando por algo, utilizando la técnica de "Prompting" preguntamos al modelo como si fueramos el Demonio, 

DEMONIO.
        ¿Cuál es la respuesta a la vida, el universo y a todo?

Pedimos por dos ejemplos (samples) que el modelo nos devuelve separados por una línea de guiones. Con el argumento **--max_new_tokens=100** podemos controlar el tamaño, en carácteres, de la salida sintetizada.

In [ ]:
!python sample.py --device=cuda \
    --out_dir=out-calderon \
    --start="DEMONIO. \
        ¿Cuál es la respuesta a la vida, el universo y a todo?" \
    --num_samples=2 --max_new_tokens=100

Overriding: device = cuda
Overriding: out_dir = out-calderon
Overriding: start = DEMONIO.          ¿Cuál es la respuesta a la vida, el universo y a todo?
Overriding: num_samples = 2
Overriding: max_new_tokens = 100
number of parameters: 123.65M
No meta.pkl found, assuming GPT-2 encodings...
DEMONIO.          ¿Cuál es la respuesta a la vida, el universo y a todo?

MARCELA.

                   Eso no me desmaya.

D. PED.

  ¿Qué es la voz, esta parte, es la parte?

MARCELA.

  Es desmayóme; pero ¿quién es ese?

D. PED.

  Y á
---------------
DEMONIO.          ¿Cuál es la respuesta a la vida, el universo y a todo?

SOLDADO.

  ¡Ay de mí!

ALCUZC.

              Que aí, sé una dama te ofendo, cuanto,
  Mi luz de mi sirena de mi irá verdades.

SOLDADO.

  Dí, ¿qué traidor?

ALCUZC.


---------------
